<a href="https://colab.research.google.com/github/VerabelGonzales/Project_Alpha_Vision/blob/main/Main_Code_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-contrib-python==3.4.2.17

# **Main Code I**

## Import Libreries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.cluster import KMeans
from collections import Counter
from skimage.color import rgb2lab, deltaE_cie76
import cv2
import os, requests
import copy
import pandas as pd
from statistics import mean
from numpy.ma.core import append
import glob
from IPython.display import Image, display
from numpy.ma.core import append
from statistics import mean
import copy
import random 
from google.colab.patches import cv2_imshow

%matplotlib inline

## Clone Our GitHub Repository

In [ ]:
!git clone https://github.com/VerabelGonzales/Project_Alpha_Vision

## Functions

In [ ]:
def show_create_folder(path):
  files_names = os.listdir(path)

  new_folder_path = "/content/Images_Dogs"
  if not os.path.exists(new_folder_path):
    os.makedirs(new_folder_path)

  cont = 0
  for file_name in files_names:
    if file_name.split(".")[-1]not in ["jpg", "png", "jpeg"]:
      continue
    image_path = path + "/" + file_name
    print(image_path)
    image = cv2.imread(image_path)
    if image is None:
      continue

    cv2.imwrite(new_folder_path +"/image" + str(cont)+".jpg", image)
    cont += 1

In [ ]:
def breed_yolo():
  # For a golden retriever
  !python /content/Project_Alpha_Vision/Detection_Breed/detect_lost_golden.py --weights /content/Project_Alpha_Vision/dog_models/golden.pt --img 416 --conf 0.1 --source  /content/Images_Dogs

  # For a Pug
  !python /content/Project_Alpha_Vision/Detection_Breed/detect_lost_pug.py --weights /content/Project_Alpha_Vision/dog_models/pug_model.pt --img 416 --conf 0.1 --source /content/Images_Dogs

  # For a German Shepherd 
  !python /content/Project_Alpha_Vision/Detection_Breed/detect_lost_german.py --weights /content/Project_Alpha_Vision/dog_models/german_shepherd_model.pt --img 416 --conf 0.1 --source /content/Images_Dogs

  # For a Chihuahua
  !python /content/Project_Alpha_Vision/Detection_Breed/detect_lost_chihuahua.py --weights /content/Project_Alpha_Vision/dog_models/chihuahua_model.pt --img 416 --conf 0.1 --source /content/Images_Dogs

  # For a Husky Siveriano
  #!python /content/Project_Alpha_Vision/Detection_Breed/detect_lost_siveriano.py --weights /content/Project_Alpha_Vision/dog_models/siberian_husky_model.pt --img 416 --conf 0.1 --source /content/Images_Dogs

  #For a Boxer
  #!python /content/Project_Alpha_Vision/Detection_Breed/detect_lost_boxer.py --weights /content/Project_Alpha_Vision/dog_models/boxer_model.pt --img 416 --conf 0.1 --source /content/Images_Dogs

  # For a San Berbardo 
  !python /content/Project_Alpha_Vision/Detection_Breed/detect_lost_sanbernardo.py --weights /content/Project_Alpha_Vision/dog_models/saint_bernard_model.pt --img 416 --conf 0.1 --source /content/Images_Dogs

  # For a Schauzer 
  !python /content/Project_Alpha_Vision/Detection_Breed/detect_lost_schnauzer.py --weights /content/Project_Alpha_Vision/dog_models/schnauzer_model.pt --img 416 --conf 0.1 --source /content/Images_Dogs

In [ ]:
def determine_breed():
  folders = ['Boxer','Chihuahua','German_Shepherd','Golden_Retriever','Pug','San_Bernardo','Schnauser','Husky_Siveriano']
  breed = np.zeros(len(folders))
  for i in range(len(folders)):
    for imageName in glob.glob('/content/Project_Alpha_Vision/Find/'+folders[i]+'/*.jpg'): #assuming JPG
      breed[i] = breed[i] + 1
      display(Image(filename=imageName, width = 300, height = 200))
      print("\n")

  max_value = None
  max_idx = None
  for idx, num in enumerate(breed):
      if (max_value is None or num > max_value):
          max_value = num
          max_idx = idx
  
  return max_value, max_idx, folders[max_idx]

In [ ]:
def RGB2HEX(color):
    return "#{:02x}{:02x}{:02x}".format(int(color[0]), int(color[1]), int(color[2]))

In [ ]:
def color_determination(imagename):
  image = cv2.imread(imagename)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  number_of_colors = 1
  modified_image = image.reshape(image.shape[0]*image.shape[1], 3)
  clf = KMeans(n_clusters = number_of_colors)
  labels = clf.fit_predict(modified_image)

  counts = Counter(labels)

  center_colors = clf.cluster_centers_
  # We get ordered colors by iterating through the keys
  ordered_colors = [center_colors[i] for i in counts.keys()]
  hex_colors = [RGB2HEX(ordered_colors[i]) for i in counts.keys()]
  rgb_colors = [ordered_colors[i] for i in counts.keys()]

  #plt.title('Color Detection', fontsize=20)
  #plt.pie(counts.values(), labels = hex_colors, colors = hex_colors)

  return rgb_colors

In [ ]:
def determine_color(n_images, breed):
  fils = int(n_images)
  colors = np.zeros((fils,3))
  cont = 0
  for imageName in glob.glob('/content/Project_Alpha_Vision/Find/'+breed+'/*.jpg'):
    colors[cont:cont+1] = color_determination(imageName)
    cont = cont + 1
  
  return colors 

In [ ]:
def found_colors_dataset(image_type, breed, RMAX, GMAX, BMAX, RMIN, GMIN, BMIN):
  
  new_folder_path = "/content/Images_Dogs_Colors"
  if not os.path.exists(new_folder_path):
    os.makedirs(new_folder_path)

  colors = np.zeros((1,3))
  cont = 0
  for imageName in glob.glob('/content/Project_Alpha_Vision/Dataset/'+image_type+'/'+breed+'/*.jpg'):
    colors[0:] = color_determination(imageName)

    image = cv2.imread(imageName)
    if colors[0,0] > RMIN-20 and colors[0,0] < RMAX+20:
      if colors[0,1] > GMIN-20 and colors[0,1] < GMAX+20:
        if colors[0,2] > BMIN-20 and colors[0,2] < BMAX+20:
          cv2.imwrite(new_folder_path +"/image" + str(cont)+".jpg", image)
          cont += 1
  
  if  cont > 0:
    for imageName2 in glob.glob(new_folder_path+'/*.jpg'):
      display(Image(filename=imageName2, width = 300, height = 200))
      print("\n")

In [ ]:
def surf_feature_extraction(training_image, query_image):
  # Convert the training image to gray scale
  training_gray = cv2.cvtColor(training_image, cv2.COLOR_BGR2GRAY)
  # Convert the query image to gray scale
  query_gray = cv2.cvtColor(query_image, cv2.COLOR_BGR2GRAY)

  # Initiate SIFT detector
  surf = cv2.xfeatures2d.SURF_create()
  # find the keypoints and descriptors with SIFT
  keypoints_train, descriptors_train = surf.detectAndCompute(training_gray, None)
  keypoints_query, descriptors_query = surf.detectAndCompute(query_gray, None)

  # FLANN parameters
  FLANN_INDEX_KDTREE = 1
  index_params = dict (algorithm = FLANN_INDEX_KDTREE, trees = 5)
  search_params = dict (checks = 30)

  flann = cv2.FlannBasedMatcher(index_params,search_params)
  matches = flann.knnMatch(descriptors_train, descriptors_query, k = 2)

  # This value defines the minimum number of matches 
  Min_Match = 5

  #This vector store all the good matches
  good = []
  for m,n in matches:
    if m.distance < 0.5*n.distance:
      good.append(m)

    # We verify if we have enough coincidences (more than 10), if so we extract the locations of the key points that coincide in both images
    if len(good) > Min_Match:
      src_pts = np.float32([ keypoints_train[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
      dst_pts = np.float32([ keypoints_query[m.trainIdx].pt for m in good ]).reshape(-1,1,2) 

      M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0) # Extract the locations of the key points that coincide in both images
      matchesMask = mask.ravel().tolist()

      h,w = training_gray.shape
      pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
      dst = cv2.perspectiveTransform(pts,M) # We do a perspective transform 

      query_gray = cv2.polylines(query_gray,[np.int32(dst)],True,255,3, cv2.LINE_AA) # We use this OpenCV function to draw several lines at the same time 

    else:
      matchesMask = None
      
    draw_params = dict(matchColor = (0,255,255),singlePointColor = None, matchesMask = matchesMask,flags = 2)
    
  result = cv2.drawMatches(training_image, keypoints_train, query_image, keypoints_query, good, None, **draw_params)
  n_matching = len(good)
  
  return result, n_matching

In [ ]:
def feature_extraction_prepros(breed):
  filenames = []
  images = []

  # The images in the github repository
  for filename in os.listdir('/content/Project_Alpha_Vision/Find/'+breed+'/'):
    if filename.split(".")[-1]not in ["jpg", "png", "jpeg"]:
      continue
    filenames.append(filename)
    image = cv2.imread('/content/Project_Alpha_Vision/Find/'+breed+'/'+filename)
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    images.append(image)

  num_images = len(images)

  # Dogs
  for i in range(num_images):
      index = i
      image = images[index]
      filename = filenames[index]

  return filenames, images, num_images

In [ ]:
def run_extraction(filenames, images, Dog_image, num_images):
  threshold = np.zeros(len(images))
  for index in range(num_images):
    filename = filenames[index]
    result_surf = surf_feature_extraction(Dog_image, images[index])
    plt.figure(figsize = (25, 50))
    plt.title(filename)
    plt.imshow(result_surf[0]),plt.show()
    threshold[index] = result_surf[1]

  return sum(threshold)

In [ ]:
def feature_extraction(filenames, images,num_images):

  new_folder_path = "/content/Feature_extraction_Posibles"
  if not os.path.exists(new_folder_path):
    os.makedirs(new_folder_path)

  cont = 0
  for imageName in glob.glob('/content/Images_Dogs_Colors/*.jpg'):
    Dog_image = cv2.imread(imageName)
    image = cv2.imread(imageName)
    Dog_image  = cv2.cvtColor(Dog_image , cv2.COLOR_BGR2RGB)

    matches = run_extraction(filenames, images, Dog_image, num_images)

    if matches > 2:
      cv2.imwrite(new_folder_path +"/image" + str(cont)+".jpg", image)
      cont += 1
  
  for imageName in glob.glob(new_folder_path+'/*.jpg'): #assuming JPG
    display(Image(filename=imageName, width = 300, height = 200))
    print("\n")

In [ ]:
def program_reset():
  folders = ['Boxer','Chihuahua','German_Shepherd','Golden_Retriever','Pug','San_Bernardo','Schnauser','Husky_Siveriano']
  for i in range(len(folders)):
    for imageName in glob.glob('/content/Project_Alpha_Vision/Find/'+folders[i]+'/*.jpg'):
      if os.path.exists(imageName):
        os.remove(imageName)
      else:
        continue 

  for imageName in glob.glob('/content/Feature_extraction_Posibles/*.jpg'):
    if os.path.exists(imageName):
      os.remove(imageName)
    else:
        continue 
  
  for imageName in glob.glob('/content/Images_Dogs/*.jpg'):
    if os.path.exists(imageName):
      os.remove(imageName)
    else:
        continue 
  
  for imageName in glob.glob('/content/Images_Dogs_Colors/*.jpg'):
    if os.path.exists(imageName):
      os.remove(imageName)
    else:
        continue 

In [ ]:
def copy_images(type_image, breed):
  for imageName in glob.glob('/content/Project_Alpha_Vision/Find/'+breed+'/*.jpg'):
    image = cv2.imread(imageName)
    cv2.imwrite("/content/Project_Alpha_Vision/Dataset/"+type_image+"/"+breed+"/"+str(random.randint(10,200000))+".jpg", image)

## Menu


In [ ]:
menu = int(input("Main Menu: \n 1- I want to search for my dog \n 2- I want to report a possible lost dog \n 0- Exit \n"))
while menu != 0:
  if menu == 1:

    images_type = "Lost_Dogs"
    path = str(input("Please enter the path of your images: "))

    show_create_folder(path)
    breed_yolo()
    dog_breed = determine_breed()

    #print('Maximum value:', dog_breed[0], "At index: ", dog_breed[1])
    #print("\n")
    print('The breed of the lost dog is:',dog_breed[2])
    print("\n")

    rgb_values = determine_color(dog_breed[0], dog_breed[2])

    print("These are the RGB values of each image: ")
    print("\n")
    print(rgb_values)
    
    RMX = max(rgb_values[0:int(dog_breed[0]),0])
    GMX = max(rgb_values[0:int(dog_breed[0]),1])
    BMX = max(rgb_values[0:int(dog_breed[0]),2])

    RMN = min(rgb_values[0:int(dog_breed[0]),0])
    GMN = min(rgb_values[0:int(dog_breed[0]),1])
    BMN = min(rgb_values[0:int(dog_breed[0]),2])

    R = mean(rgb_values[0:int(dog_breed[0]),0])
    G = mean(rgb_values[0:int(dog_breed[0]),1])
    B = mean(rgb_values[0:int(dog_breed[0]),2])
    print("\n")
    print("Dogs with similar colors: ")
    found_colors_dataset(images_type, dog_breed[2], RMX, GMX, BMX, RMN, GMN, BMN)

    print('\n')
    print('These pets were found, they have similar characteristics to those of your pet: ')
    characteristics = feature_extraction_prepros(dog_breed[2])
    feature_extraction(characteristics[0], characteristics[1], characteristics[2])

    print('\n')
    print('Have you found your pet? ')
    options = int(input("\n 1- Press 1 if you found it: \n 2- Press 2 if you did not find it: \n"))
    save_images_type = "Wanted_Dogs"
    while options != 0:
      if options == 1:
        program_reset()
        break
      elif options == 2:
        copy_images(save_images_type, dog_breed[2])
        program_reset()
        break

    break    

  elif menu == 2:

    images_type = "Wanted_Dogs"
    path = str(input("Please enter the path of your images: "))

    show_create_folder(path)
    breed_yolo()
    dog_breed = determine_breed()

    #print('Maximum value:', dog_breed[0], "At index: ", dog_breed[1])
    #print("\n")
    print('The breed of the posible lost dog is:',dog_breed[2])
    print("\n")

    rgb_values = determine_color(dog_breed[0], dog_breed[2])

    print("These are the RGB values of each image")
    print("\n")
    print(rgb_values)
    
    RMX = max(rgb_values[0:int(dog_breed[0]),0])
    GMX = max(rgb_values[0:int(dog_breed[0]),1])
    BMX = max(rgb_values[0:int(dog_breed[0]),2])

    RMN = min(rgb_values[0:int(dog_breed[0]),0])
    GMN = min(rgb_values[0:int(dog_breed[0]),1])
    BMN = min(rgb_values[0:int(dog_breed[0]),2])

    R = mean(rgb_values[0:int(dog_breed[0]),0])
    G = mean(rgb_values[0:int(dog_breed[0]),1])
    B = mean(rgb_values[0:int(dog_breed[0]),2])
    print("\n")
    print("Dogs with similar colors: ")
    found_colors_dataset(images_type, dog_breed[2], RMX, GMX, BMX, RMN, GMN, BMN)

    print('\n')
    print('These pets were found, they have similar characteristics to the possible lost dog: ')
    characteristics = feature_extraction_prepros(dog_breed[2])
    feature_extraction(characteristics[0], characteristics[1], characteristics[2])

    print('\n')
    print('Was the owner of the pet found? ')
    options = int(input("\n 1- Press 1 if the owner found his pet \n 2- Press 2 if the owner did not find his pet \n"))
    save_images_type = "Lost_Dogs"
    while options != 0:
      if options == 1:
        program_reset()
        break
      elif options == 2:
        copy_images(save_images_type, dog_breed[2])
        program_reset()
        break

    break 
  else:
    print("Please enter the correct option")